In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, precision_score, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'numpy'

In [2]:
# Получаем данные
dataset = pd.read_csv("../data/doctor_vs_symptoms.csv", sep=";", encoding="cp1251")

In [4]:
half_symptoms_data = [
    ['насморк эпизодами','чихание эпизодами','дрожь','озноб','слезотечение из глаз эпизодами'],
    ['болезненное мочеиспускание','гнойные выделения из уретры','покраснение и отёк уретры','гнойные выделения из влагалища'],
    ['боль в верхней части живота','изжога','язвы на языке','рвота'],
    ['бледная кожа','мелкие кровоизлияния в кожу','увеличенные лимфатические узлы','увеличение дёсен'],
    ['боль внизу живота','кровянистые выделения из влагалища','изменение характера выделений из влагалища'],
    ['кожный зуд','кожная сыпь','общая слабость','повышение температуры тела'],
    ['кожная сыпь','озноб','боль в суставах','рвота','общая слабость','повышение температуры тела'],
    ['боль в груди','сжимающая боль в груди','одышка','чувство нехватки воздуха','головокружение'],
    ['боль в молочных железах','боль в молочных железах незадолго до менструации','появление уплотнений или образований в молочных железах'],
    ['боль в спине','боль в шее','ограничение движений в спине','ограничение движений в шее','головокружение'],
    ['боль в пояснице','повышенное давление','высокий уровень сахара в крови','увеличенный объём мочи','частые ночные походы в туалет'],
    ['кожный зуд у глаз','покраснение глаз','шелушение кожи'],
    ['чихание','озноб','общая слабость','кашель','повышение температуры тела','головная боль','увеличенные лимфатические узлы','мокрота'],
    ['появление крови в стуле','повышенный аппетит','потеря аппетита','диарея'],
    ['галлюцинации','убеждения не поддающиеся логике','несвязная или нелогичная речь'],
    ['приступы удушья','сильный кашель','одышка при физической нагрузке'],
    ['боль в конечностях','покраснение на коже','опухание сустава','уменьшение подвижности в суставе'],
    ['зубная боль','повышение температуры тела','кровоточивость десны','припухлость десны'],
    ['боль в пояснице','боль в нижней части живота','частое мочеиспускание','болезненное мочеиспускание'],
    ['покраснение около вены на ноге','боль около вены на ноге','отёки на ногах'],
    ['рана на коже','боль от раны','кровотечение из раны'],
    ['общая слабость','увеличение веса','ощущение холода даже в тепле','перепады настроения','ломкие ногти','ломкие волосы']
]

half_symptoms_answers = [
    'Аллерголог', 'Венеролог','Гастроэнтеролог', 'Гематолог', 'Гинеколог', 'Дерматолог', 'Инфекционист', 'Кардиолог', 'Маммолог',
    'Невролог', 'Нефролог', 'Окулист', 'Отоларинголог', 'Проктолог', 'Психиатр', 'Пульмонолог', 'Ревматолог', 'Стоматолог',
    'Уролог', 'Флеболог', 'Хирург', 'Эндокринолог'
]

unique_symptoms_data = [
    ['насморк эпизодами','чихание эпизодами','слезотечение из глаз эпизодами', 'отёк эпизодами'],
    ['гнойные выделения из уретры','покраснение и отёк уретры','гнойные выделения из влагалища', 'боль в яичках'],
    ['впавшие глаза', 'чувство тяжести в верхней части живота', 'быстрая утомляемость','язвы на языке'],
    ['бледная кожа', 'мелкие кровоизлияния в кожу', 'увеличение дёсен', 'отёки на лице'],
    ['изменение характера выделений из влагалища', 'выделения из влагалища с гнилостным запахом', 'бесплодие', 'увеличение живота'],
    ['угри', 'рубцы на коже', 'узловые кожные высыпания', 'серебристо-белые высыпания'],
    ['слабый иммунитет', 'увеличение затылочных лимфоузлов', 'появление белых пятен во рту', 'исчезновение сыпи через несколько дней'],
    ['сжимающая боль в груди', 'перебои в работе сердца', 'приступы боли в груди', 'боль проходит после приёма нитроглицерина'],
    ['боль в молочных железах', 'боль в молочных железах незадолго до менструации', 'появление уплотнений или образований в молочных железах', 'выделения из сосков от различных оттенков жёлтого до коричневого и серого'],
    ['приступы головной боли', 'слабость в руках', 'онемение в руках', 'ограничение движений в спине'],
    ['боль независит от положения или движений', 'увеличенный объём мочи', 'частые ночные походы в туалет', 'высокий уровень сахара в крови'],
    ['ухудшение зрения', 'расплывчатость объектов', 'гнойные выделения из глаз', 'нарушение цветоощущения'],
    ['першение в горле', 'боль в области переносицы', 'желто-зеленые выделения из носа', 'отсутствие голоса'],
    ['появление крови в стуле', 'боль во время дефекации', 'боль в анальной области', 'раздражение в анусе'],
    ['убеждения не поддающиеся логике', 'галлюцинации', 'отсутствие критики к своему состоянию', 'навязчивые мысли и действия'],
    ['приступы удушья', 'свистящие хрипы', 'боль в груди при кашле', 'свистящее дыхание'],
    ['опухание сустава', 'боль в суставе', 'уменьшение подвижности в суставе', 'скованность в движениях'],
    ['зубная боль', 'зубная боль усиливается от горячего', 'кровоточивость десны', 'зубная боль усиливается от холодного'],
    ['дискомфорт в мочевом пузыре', 'наличие крови в моче и сперме', 'чувство что мочевой пузырь опорожнён не полностью', 'затрудненное мочеиспускание'],
    ['тяжесть в ногах', 'красные, синие или фиолетовые сосудистые образования', 'боль около вены на ноге', 'покраснение около вены на ноге'],
    ['кровотечение из раны', 'сильная боль в области кости', 'сильная боль при движении конечностями', 'видимая деформация кости'],
    ['сильная жажда', 'ощущение холода даже в тепле', 'ломкие ногти', 'ломкие волосы']
]

unique_symptoms_answers = [
    'Аллерголог', 'Венеролог','Гастроэнтеролог', 'Гематолог', 'Гинеколог', 'Дерматолог', 'Инфекционист', 'Кардиолог', 'Маммолог',
    'Невролог', 'Нефролог', 'Окулист', 'Отоларинголог', 'Проктолог', 'Психиатр', 'Пульмонолог', 'Ревматолог', 'Стоматолог',
    'Уролог', 'Флеболог', 'Хирург', 'Эндокринолог'
]

random_symptoms_data =[
    ['кашель', 'головная боль', 'общая слабость', 'озноб', 'потеря аппетита', 'зуд в горле', 'головокружение', 'мокрота'],
    ['боль в груди','сжимающая боль в груди','одышка','учащенный пульс','мушки перед глазами','холодный пот','повышенное давление'],
    ['запор', 'боль во время дефекации', 'появление крови в стуле', 'обезвоживание', 'диарея', 'раздражение в анусе'],
    ['раздражение в анусе'],
    ['выделение гноя из уха'],
    ['утрата брюшных рефлексов'],
    ['изменение характера выделений из влагалища']
]

# ['Невролог' 'Кардиолог' 'Проктолог' 'Невролог' 'Отоларинголог' 'Невролог'
#  'Гинеколог']

test_data = [
    ['боль в груди','сжимающая боль в груди','одышка','учащенный пульс','мушки перед глазами','холодный пот','повышенное давление'],
    ['сжимающая боль в груди', 'боль в груди','мушки перед глазами','одышка','учащенный пульс','холодный пот','повышенное давление']
]



In [5]:
# Собираем все симптомы через запятую в один столбец
dataset['Симптомы'] = dataset[["Симптом 1","Симптом 2","Симптом 3","Симптом 4","Симптом 5","Симптом 6","Симптом 7","Симптом 8","Симптом 9",
                               "Симптом 10","Симптом 11","Симптом 12","Симптом 13","Симптом 14","Симптом 15"]].agg(lambda symp: ';'.join(symp.dropna().astype(str)), axis=1)

dataset = dataset[['Специалист', 'Симптомы']]

# Разбиваем симптомы по запятым
dataset['Симптомы'] = dataset['Симптомы'].apply(lambda x: [symptom.strip() for symptom in x.split(';')])

In [6]:
#dataset['Симптомы'][0]

In [7]:
# Кодирование симптомов
mlb = MultiLabelBinarizer()
data_x = mlb.fit_transform(dataset['Симптомы']) # 313 уникальных симптомов

In [8]:
#data_x[0]

In [9]:
# Выделяем уникальных специалистов
#unique_specialists = dataset['Специалист'].drop_duplicates().tolist() # 23 уникальных специалиста
#unique_spec_df = pd.DataFrame({ 'Специалист': unique_specialists })

In [10]:
# Обучаем на уникальных и кодируем специалистов из даты
#encoder = OneHotEncoder()
##encoder.fit(unique_spec_df)
#data_y = encoder.transform(dataset[['Специалист']])

In [11]:
# # Кодируем специалистов
# encoder = OneHotEncoder(handle_unknown='ignore')
# data_y = encoder.fit_transform(dataset[['Специалист']]).toarray() # 22 уникальных специалиста

In [12]:
# Кодируем специалистов
lbe = LabelEncoder()
data_y = lbe.fit_transform(dataset['Специалист']) # 22 уникальных специалиста

In [13]:
#data_y

In [14]:
# Разделение данных на обучающую и тестовую выборки
#x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.22, random_state=42, stratify=data_y)

In [15]:
# smoten = SMOTEN(random_state=42, k_neighbors=1)
# x_resampled, y_resampled = smoten.fit_resample(data_x, data_y)

In [16]:
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(data_x, data_y)

In [17]:
#resaml_y = lbe.inverse_transform(y_resampled)
#resaml_x = mlb.inverse_transform(x_resampled)
#ind = 114
#print(resaml_y[ind])
#resaml_x[ind]

In [18]:
# count = 0
# for i in range(len(resaml_y)):
#     if resaml_y[i] == 'Психиатр':
#         count = count + 1


# print(count)

In [19]:
# Обучение модели SVC
c_ = 0.2 # 0.2
gamma_ = 'scale' #'scale'
kernel_ = 'linear' #'linear'
model_svc = SVC(C=c_, gamma=gamma_, kernel=kernel_,)

print("C=", c_, "gamma=", gamma_, "kernel=", kernel_)

# Стратифицированный multi-label KFold кросс валидация
#cv_ = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=42)

cv_ = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Оценка модели
scorer = make_scorer(accuracy_score)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя accuracy:", np.mean(scores))

scorer = make_scorer(f1_score, average='micro', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (micro):", np.mean(scores))

scorer = make_scorer(f1_score, average='macro', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (macro):", np.mean(scores))

scorer = make_scorer(f1_score, average='weighted', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (weighted):", np.mean(scores))

scorer = make_scorer(precision_score, average='micro', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_micro:", np.mean(scores))

scorer = make_scorer(precision_score, average='macro', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_macro:", np.mean(scores))

scorer = make_scorer(precision_score, average='weighted', zero_division=0)
scores = cross_val_score(model_svc, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_weighted:", np.mean(scores))

# Обучение модели на полных данных
model_svc.fit(x_resampled, y_resampled)

C= 0.2 gamma= scale kernel= linear
Средняя accuracy: 0.8943589743589744
Средняя F1 (micro): 0.8943589743589744
Средняя F1 (macro): 0.8751948051948052
Средняя F1 (weighted): 0.8803467643467645
Средняя precision_micro: 0.8943589743589744
Средняя precision_macro: 0.8915151515151514
Средняя precision_weighted: 0.9001752136752137


SVC(C=0.2, kernel='linear')

In [20]:
# param_grid = {
#     'estimator__C': [0.1, 1, 10],
#     'estimator__gamma': ['scale', 'auto', 0.01, 0.1, 1],
#     'estimator__kernel': ['rbf', 'linear', 'poly']
# }

# # Поиск по сетке
# grid_search = GridSearchCV(model_svc, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(data_x, data_y)

# # Лучшие параметры и результат
# print("Лучшая конфигурация:", grid_search.best_params_)
# print("Лучшая точность:", grid_search.best_score_)

In [21]:
half_symp_transform = mlb.transform(half_symptoms_data)
half_symp_pred_encode = model_svc.predict(half_symp_transform)

#len(x_my_data[0])
half_symp_pred = lbe.inverse_transform(half_symp_pred_encode)
#print(half_symp_pred)

In [22]:
good_ans_half_symp = 0

for i in range(0, len(half_symptoms_answers)):
    if half_symp_pred[i] == half_symptoms_answers[i]:
        good_ans_half_symp = good_ans_half_symp + 1

print("Правильно отвечено:", good_ans_half_symp, "Всего вопросов:", len(half_symptoms_answers))

Правильно отвечено: 21 Всего вопросов: 22


In [23]:
#half_symptoms_answers

In [24]:
unique_symp_transform = mlb.transform(unique_symptoms_data)
unique_symp_pred_encode = model_svc.predict(unique_symp_transform)

#len(x_my_data[0])
unique_symp_pred = lbe.inverse_transform(unique_symp_pred_encode)
#print(unique_symp_pred)

In [25]:
good_ans_unique_symp = 0

for i in range(0, len(unique_symptoms_answers)):
    if unique_symp_pred[i] == unique_symptoms_answers[i]:
        good_ans_unique_symp = good_ans_unique_symp + 1

print("Правильно отвечено:", good_ans_unique_symp, "Всего вопросов:", len(unique_symptoms_answers))

Правильно отвечено: 17 Всего вопросов: 22


In [26]:
random_symp_transform = mlb.transform(random_symptoms_data)
random_symp_pred_encode = model_svc.predict(random_symp_transform)

random_symp_pred = lbe.inverse_transform(random_symp_pred_encode)
print(random_symp_pred)

['Невролог' 'Кардиолог' 'Проктолог' 'Невролог' 'Отоларинголог' 'Невролог'
 'Невролог']


In [27]:
# Обучение модели Random Forest
model_rf = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf=2, random_state=42)

cv_ = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Оценка модели
scorer = make_scorer(accuracy_score)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя accuracy:", np.mean(scores))

scorer = make_scorer(f1_score, average='micro', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (micro):", np.mean(scores))

scorer = make_scorer(f1_score, average='macro', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (macro):", np.mean(scores))

scorer = make_scorer(f1_score, average='weighted', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (weighted):", np.mean(scores))

scorer = make_scorer(precision_score, average='micro', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_micro:", np.mean(scores))

scorer = make_scorer(precision_score, average='macro', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_macro:", np.mean(scores))

scorer = make_scorer(precision_score, average='weighted', zero_division=0)
scores = cross_val_score(model_rf, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_weighted:", np.mean(scores))

# Обучение модели на полных данных
model_rf.fit(x_resampled, y_resampled)

Средняя accuracy: 0.8587179487179487
Средняя F1 (micro): 0.8587179487179487
Средняя F1 (macro): 0.8324242424242424
Средняя F1 (weighted): 0.8363675213675213
Средняя precision_micro: 0.8587179487179487
Средняя precision_macro: 0.8409090909090908
Средняя precision_weighted: 0.848258547008547


RandomForestClassifier(max_depth=20, min_samples_leaf=2, random_state=42)

In [28]:
half_symp_transform = mlb.transform(half_symptoms_data)
half_symp_pred_encode = model_rf.predict(half_symp_transform)

#len(x_my_data[0])
half_symp_pred = lbe.inverse_transform(half_symp_pred_encode)
#print(half_symp_pred)

In [29]:
good_ans_half_symp = 0

for i in range(0, len(half_symptoms_answers)):
    if half_symp_pred[i] == half_symptoms_answers[i]:
        good_ans_half_symp = good_ans_half_symp + 1

print("Правильно отвечено:", good_ans_half_symp, "Всего вопросов:", len(half_symptoms_answers))

Правильно отвечено: 21 Всего вопросов: 22


In [30]:
unique_symp_transform = mlb.transform(unique_symptoms_data)
unique_symp_pred_encode = model_rf.predict(unique_symp_transform)

#len(x_my_data[0])
unique_symp_pred = lbe.inverse_transform(unique_symp_pred_encode)
#print(unique_symp_pred)

In [31]:
good_ans_unique_symp = 0

for i in range(0, len(unique_symptoms_answers)):
    if unique_symp_pred[i] == unique_symptoms_answers[i]:
        good_ans_unique_symp = good_ans_unique_symp + 1

print("Правильно отвечено:", good_ans_unique_symp, "Всего вопросов:", len(unique_symptoms_answers))

Правильно отвечено: 18 Всего вопросов: 22


In [32]:
random_symp_transform = mlb.transform(random_symptoms_data)
random_symp_pred_encode = model_rf.predict(random_symp_transform)

random_symp_pred = lbe.inverse_transform(random_symp_pred_encode)
print(random_symp_pred)

['Отоларинголог' 'Кардиолог' 'Проктолог' 'Невролог' 'Невролог' 'Невролог'
 'Гинеколог']


In [37]:
# Количество классов
num_classes = len(set(y_resampled))

# Модель XGBoost
model_xgb = XGBClassifier(
    objective='multi:softprob',
    num_class=num_classes,
    eval_metric='mlogloss',
    random_state=42,
)

cv_ = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Оценка модели
scorer = make_scorer(accuracy_score)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя accuracy:", np.mean(scores))

scorer = make_scorer(f1_score, average='micro', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (micro):", np.mean(scores))

scorer = make_scorer(f1_score, average='macro', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (macro):", np.mean(scores))

scorer = make_scorer(f1_score, average='weighted', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя F1 (weighted):", np.mean(scores))

scorer = make_scorer(precision_score, average='micro', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_micro:", np.mean(scores))

scorer = make_scorer(precision_score, average='macro', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_macro:", np.mean(scores))

scorer = make_scorer(precision_score, average='weighted', zero_division=0)
scores = cross_val_score(model_xgb, x_resampled, y_resampled, scoring=scorer, cv=cv_)
print("Средняя precision_weighted:", np.mean(scores))

# Обучение на полных данных
model_xgb.fit(x_resampled, y_resampled)

Средняя accuracy: 0.8084615384615385
Средняя F1 (micro): 0.8084615384615385
Средняя F1 (macro): 0.7984271284271284
Средняя F1 (weighted): 0.7988937728937728
Средняя precision_micro: 0.8084615384615385
Средняя precision_macro: 0.8320454545454545
Средняя precision_weighted: 0.8384284188034188


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=22, ...)

In [38]:
half_symp_transform = mlb.transform(half_symptoms_data)
half_symp_pred_encode = model_xgb.predict(half_symp_transform)

#len(x_my_data[0])
half_symp_pred = lbe.inverse_transform(half_symp_pred_encode)
#print(half_symp_pred)

In [39]:
good_ans_half_symp = 0

for i in range(0, len(half_symptoms_answers)):
    if half_symp_pred[i] == half_symptoms_answers[i]:
        good_ans_half_symp = good_ans_half_symp + 1

print("Правильно отвечено:", good_ans_half_symp, "Всего вопросов:", len(half_symptoms_answers))

Правильно отвечено: 20 Всего вопросов: 22


In [40]:
unique_symp_transform = mlb.transform(unique_symptoms_data)
unique_symp_pred_encode = model_xgb.predict(unique_symp_transform)

#len(x_my_data[0])
unique_symp_pred = lbe.inverse_transform(unique_symp_pred_encode)
#print(unique_symp_pred)

In [41]:
good_ans_unique_symp = 0

for i in range(0, len(unique_symptoms_answers)):
    if unique_symp_pred[i] == unique_symptoms_answers[i]:
        good_ans_unique_symp = good_ans_unique_symp + 1

print("Правильно отвечено:", good_ans_unique_symp, "Всего вопросов:", len(unique_symptoms_answers))

Правильно отвечено: 11 Всего вопросов: 22


In [42]:
random_symp_transform = mlb.transform(random_symptoms_data)
random_symp_pred_encode = model_xgb.predict(random_symp_transform)

random_symp_pred = lbe.inverse_transform(random_symp_pred_encode)
print(random_symp_pred)

['Невролог' 'Кардиолог' 'Проктолог' 'Хирург' 'Хирург' 'Хирург' 'Гинеколог']
